# Comissão do Setor de Cobranças da Imobiliária

Esse é o notebook da automação do setor de cobrança de uma imobiliária.

Para explicações detalhadas, acesse o nosso site: <a href="igorseixas.wordpress.com">igorseixas.wordpress.com</a>

No site temos disponibilizado a base de dados para treinamento, tendo em vista que vocês não conseguirão acessar o sistema da imobiliária.

### 1. Importando todas as bibliotecas que serão usadas

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
import os
import shutil
import pandas as pd
from datetime import datetime
import yagmail

### 2. Escolha do período da comissão pelo usuário e quantidade de imóveis que foram rescindidos o contrato no período

In [ ]:
periodo_inicio = input('Data do início: ')
periodo_fim = input('Data do fim: ')
distratos = input('Quantos contratos recindidos no período: ')

## Baixando a base de dados

### 3. Definindo paramêtros do Chromedrive

In [ ]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={user_agent}')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)

Todos os options foram necessários para que o site que contem a base de dados não bloqueasse o acesso via selenium

### 4. Entrando no site

In [ ]:
driver.get('http://signin.ingaia.com.br/?provider=locacao')
time.sleep(5)

### 5. Fazendo o login

In [ ]:
#encontrando os campos do formulário pelo id
form_email = driver.find_element_by_id('email')
form_senha = driver.find_element_by_id('password')
logar = driver.find_element_by_id('submitButton')

#completando os campos
form_email.send_keys('email_do_login')
time.sleep(3)
form_senha.send_keys('senha_de_acesso')
time.sleep(2)
logar.click()
time.sleep(20)

### 6. Buscando o relatório

In [ ]:
empresa = driver.find_element_by_id('bootstrapmenu-tabEmpresa')
empresa.click()
time.sleep(2)
relatorio = driver.find_element_by_link_text('Relatórios')
relatorio.click()
time.sleep(2)

In [ ]:
composicao_cobranca = driver.find_element_by_link_text('Composição das cobranças por liquidação (104C)')
composicao_cobranca.click()
time.sleep(2)

In [ ]:
mes = driver.find_element_by_id('LIQUIDACAO')
mes.click()
mes.send_keys('s')
mes.click()
time.sleep(2)
data_inicial = driver.find_element_by_name('LIQUIDACAO_INICIO')
data_inicial.click()
data_inicial.send_keys(Keys.BACKSPACE * 10)
time.sleep(2)
data_inicial.send_keys(periodo_inicio)
time.sleep(2)
data_final = driver.find_element_by_name('LIQUIDACAO_FIM')
data_final.click()
time.sleep(2)
data_final.send_keys(Keys.BACKSPACE * 10)
data_final.send_keys(periodo_fim)
data_final.send_keys(Keys.ENTER)
time.sleep(2)

In [ ]:
exportar = driver.find_element_by_css_selector('button.btnPadraoExportarRelatorios')
exportar.click()
time.sleep(1)
exp_excel = driver.find_element_by_id('btnExportacaoExcel')
exp_excel.click()
time.sleep(30)


### 7. Descobrindo o nome do ultimo arquivo baixado

In [ ]:
#from pathlib import Path
#import os

caminho = Path('/Users/igorseixas/Downloads/')
arquivos = sorted(Path(caminho).iterdir(), key=os.path.getmtime, reverse=True)

for arquivo in arquivos:
    arquivo_str = str(arquivo)
    nome_arquivo = arquivo_str[28:]
    break

time.sleep(1)


### 8. Copiando o ultimo arquivo na pasta correta

In [ ]:
#import shutil

arquivo_copiar = Path(f'/Users/igorseixas/Downloads/{nome_arquivo}')
arquivo_colar = Path(f'/Users/igorseixas/PycharmProjects/Comissao_Cobranca_Automacao/{nome_arquivo}')
shutil.copy2(arquivo_copiar, arquivo_colar)

## Iniciando o tratamento da base de dados

### 9. Criando o dataframe

In [ ]:
comissao_df = pd.read_excel(nome_arquivo)

## 10. Tratando o dataframe

#### 10.1. Nomeando as colunas

In [ ]:
comissao_df.columns = comissao_df.iloc[2]


#### 10.2. Selecionando as colunas que interessam

In [ ]:
comissao_df = comissao_df[['Nome', 'Descrição', 'Vencimento', 'Liquidação', 'Valor']]


#### 10.3. Filtrar apenas as linhas onde tenha Multa

In [ ]:
multas_df = comissao_df[comissao_df['Descrição'] == 'Multa']


#### 10.4. Criando novas colunas para Dias de Atraso e Valor Comissão

In [ ]:
multas_df = multas_df.assign(Atraso=0)
multas_df = multas_df.assign(Comissão=0)

#### 10.5. Calcular quantos dias de atraso e adicionar a informação na coluna


In [ ]:
for i, data in enumerate(multas_df['Vencimento']):
    date_format = "%d/%m/%Y"
    a = datetime.strptime(multas_df.iloc[i, 2], date_format)
    b = datetime.strptime(multas_df.iloc[i, 3], date_format)
    delta = b - a
    multas_df.iloc[i, 5] = delta.days  # <- resultado em dias
#     ^ substitui o valor pelo resultado de dias

#### 10.6. Calcular o valor da comissão com base na quantidade de dias em atraso

In [ ]:
for i, dias in enumerate(multas_df['Atraso']):
    multas_df.iloc[i, 4] = multas_df.iloc[i, 4].replace(',', '.') #tirando as virgulas e trocando por .
    multas_df.iloc[i, 4] = float(multas_df.iloc[i, 4]) #transformando a string em float
    if dias <= 5:
        pass
    elif dias > 5 and dias <= 15:
        multas_df.iloc[i, 6] = multas_df.iloc[i, 4] * 0.25
    elif dias > 15 and dias <= 30:
        multas_df.iloc[i, 6] = multas_df.iloc[i, 4] * 0.15
    else:
        multas_df.iloc[i, 6] = multas_df.iloc[i, 4] * 0.10


#### 10.7. Adicionando a linha total com o valor do total da comissão.

In [ ]:
final_df = len(multas_df['Nome']) + 1
soma_comissao = multas_df['Comissão'].sum()
multas_df.loc[final_df] = ['Total', '', '', '', '', '', f'R$ {soma_comissao:.2f}']


#### 10.8. Adicionando a linha com a soma dos contratos rescindidos

In [ ]:
comissao_rescindidos = 50
total_rescindidos = distratos * comissao_rescindidos
final_df = len(multas_df['Nome']) + 1
multas_df.loc[final_df] = ['Rescindidos no mês', '', '', '', f'R$ {comissao_rescindidos:.2f}', distratos , f'R$ {total_rescindidos:.2f}']


#### 10.9. Adicionando o Total Geral

In [ ]:
final_df = len(multas_df['Nome']) + 1
total_geral = soma_comissao + total_rescindidos
multas_df.loc[final_df] = ['TOTAL GERAL', '', '', '', '', '' , f'R$ {total_geral:.2f}']
display(multas_df)

## 11. Exportando o dataframe como planilha Excel

In [ ]:
multas_df.to_excel('Comissao Cobrança.xlsx')

## 12. Enviando a planilha por email

In [ ]:
email = 'email@do_remetente.com'
destinatario = ['email@do_destinatario1.com', 'email@do_destinatario2.com']

usuario = yagmail.SMTP(user=email, password='senha_do_email')

usuario.send(to=destinatario, subject='Comissão da cobrança', contents=f'Oi,\nSegue em anexo a comissão do setor de cobrança.\nPeríodo: {periodo_inicio} a {periodo_fim}\n\n Atenciosamente,', attachments='Comissao Cobrança.xlsx')

print(f'Email enviado de {email} para {destinatario}.')

